In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['KMP_WARNINGS'] = 'off'
import sys
import git

import uproot as ut
import awkward as ak
import numpy as np
import math
import vector
import sympy as sp

import re
from tqdm import tqdm
import timeit
import re

sys.path.append( git.Repo('.', search_parent_directories=True).working_tree_dir )
from utils import *

In [14]:
pairing='pn_mindiag'
treekwargs = dict(
    use_gen=False,
    treename='Events',
    normalization=None,
)

f_pattern = '/eos/user/e/ekoenig/Ntuples/NanoHH4b/pn_mindiag_sig_2018_0L/mc/ggHH4b_tree.root'
f_sig = f_pattern.format(pairing=pairing)
signal = ObjIter([Tree( fc.fs.cernbox.glob(f_sig, with_path=True), **treekwargs)])

100%|██████████| 1/1 [00:02<00:00,  2.19s/it]


In [16]:
signal.apply(lambda t: t.reweight(t.genWeight * t.xsecWeight))

[None]